In [1]:
from exif_tagger.database import FaceDatabase,UNKNOWN_PERSON,Face,Person,Picture
from exif_tagger.face_recognition import FaceRecognitionModel
from pathlib import Path
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import PIL.Image

In [2]:
DEFAULT_DB_PATH = Path("/home/pierre/repositories/exif-tagger/db/face-database.sqlite")
DEFAULT_UNKNOWN_EMBEDDING_PATH = Path(
    "/home/pierre/repositories/exif-tagger/res/embeddings_vg2.npy"
)
DEFAULT_FACE_RECOGNITION_PATH = Path(
    "/home/pierre/repositories/exif-tagger/models/face-recognition.joblib"
)

db = FaceDatabase(DEFAULT_DB_PATH)
fc = FaceRecognitionModel.from_file(model_filename=DEFAULT_FACE_RECOGNITION_PATH)

Running on device: cpu


In [3]:

fc.detect_faces(PIL.Image.open("/home/pierre/repositories/exif-tagger/test/test_big/PXL_20231109_034149044.jpg"))

Detecting faces...
Detect boxes proba: [0.9999918937683105]
Extracting faces...
Generating embeddings...
1 torch.Size([3, 160, 160])
torch.Size([1, 3, 160, 160])
tensor([[[-0.3242, -0.3008, -0.2852,  ..., -0.1680, -0.1055, -0.0820],
         [-0.2773, -0.2461, -0.2930,  ..., -0.1602, -0.1289, -0.1445],
         [-0.1367, -0.2148, -0.2930,  ..., -0.1836, -0.1289, -0.2070],
         ...,
         [ 0.7305,  0.6133,  0.5820,  ..., -0.8086, -0.8867, -0.8789],
         [ 0.6992,  0.6289,  0.5820,  ..., -0.4336, -0.3633, -0.4727],
         [ 0.6992,  0.6445,  0.6133,  ..., -0.3320, -0.4180, -0.4570]],

        [[-0.5742, -0.5508, -0.5430,  ..., -0.3945, -0.3477, -0.3242],
         [-0.5273, -0.4961, -0.5430,  ..., -0.3945, -0.3711, -0.3867],
         [-0.3867, -0.4648, -0.5430,  ..., -0.4180, -0.3711, -0.4492],
         ...,
         [ 0.3945,  0.2617,  0.1992,  ..., -0.8086, -0.8867, -0.8789],
         [ 0.3711,  0.2695,  0.2070,  ..., -0.4336, -0.3711, -0.4727],
         [ 0.3555,  0.2773,

[Face(person_id=1, picture_id=1, confirmed=False, bbox=[771, 589, 1256, 1225], embedding=array([-1.42054381e-02, -3.79921310e-02,  4.08132710e-02,  3.02348509e-02,
        -4.36484441e-02, -4.09115218e-02, -1.12212971e-02, -4.21509445e-02,
         4.30460423e-02, -8.99925157e-02,  4.24391066e-04,  1.65135488e-02,
        -4.62289304e-02,  1.65125243e-02, -3.16373520e-02,  6.17406927e-02,
         3.74987200e-02,  1.86104812e-02,  3.11255828e-02,  7.17059802e-03,
         5.35850488e-02, -7.88270682e-02,  3.31777856e-02, -3.01834885e-02,
        -3.56592201e-02, -3.86767238e-02,  4.19465490e-02,  4.55231033e-02,
        -3.20095243e-03,  6.84620515e-02,  6.43053576e-02, -6.62508532e-02,
        -2.18569078e-02, -1.74887211e-03,  8.85051861e-03,  1.89253781e-02,
         1.24245288e-03,  3.88499512e-03, -1.47541836e-02,  4.51374166e-02,
         2.43383124e-02, -2.67278720e-02,  4.23564501e-02,  2.19335034e-02,
         2.23069098e-02, -5.44971526e-02, -3.54916044e-02, -9.26223770e-03,


In [ ]:
from exif_tagger.embeddings import generate_embeddings
embs,names = generate_embeddings("/home/pierre/Downloads/VGG-Face2/data/vggface2_test/small_test")

In [ ]:
unknown_embeddings = np.load(DEFAULT_UNKNOWN_EMBEDDING_PATH)
unknown_embeddings_train = unknown_embeddings[:len(unknown_embeddings)//2]
unknown_embeddings_test = unknown_embeddings[len(unknown_embeddings)//2:]
unknown_embeddings.shape

In [ ]:
db.getPersons()

In [ ]:
pierre = db.getPersonsByNameSurname("Pierre", "Louvart")[0]
pierre_embeddings = np.array(db.getEmbeddingsForPerson(pierre.id))
pierre_embeddings.shape
pierre_embeddings_train = pierre_embeddings[::2]
pierre_embeddings_test = pierre_embeddings[1::2]
pierre_embeddings.shape

In [ ]:
kerian = db.getPersonsByNameSurname("Kerian", "Tiennot")[0]
kerian_embeddings = np.array(db.getEmbeddingsForPerson(kerian.id))
kerian_embeddings.shape
kerian_embeddings_train = kerian_embeddings[::2]
kerian_embeddings_test = kerian_embeddings[1::2]
kerian_embeddings.shape

In [ ]:
people_embeddings = np.array(db.getEmbeddingsForPerson(UNKNOWN_PERSON.id))
people_embeddings.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

X = np.concatenate([unknown_embeddings_train, kerian_embeddings_train])
Y = np.concatenate([np.zeros(len(unknown_embeddings_train)), 1*np.ones(len(kerian_embeddings_train))])
# X = np.concatenate([unknown_embeddings_train, pierre_embeddings_train, kerian_embeddings_train])
# Y = np.concatenate([np.zeros(len(unknown_embeddings_train)), np.ones(len(pierre_embeddings_train)), 2*np.ones(len(kerian_embeddings_train))])


model.fit(X, Y)
model

In [ ]:
np.mean(
    model.predict(X[::50]) == Y[::50]
)

In [ ]:
model.predict(pierre_embeddings_test)

In [ ]:
model.predict(pierre_embeddings_train)

In [ ]:
model.predict(kerian_embeddings_test)

In [ ]:
model.predict(unknown_embeddings_train[::100])

In [ ]:
X = np.concatenate([kerian_embeddings, embs, pierre_embeddings[:20]])
print(pierre_embeddings.shape)
print(embs.shape)
print(kerian_embeddings.shape)
plt.figure(figsize=(16, 8))
plt.imshow(X)

In [ ]:
X = np.concatenate([embs, unknown_embeddings[::100]])
print(pierre_embeddings.shape)
print(embs.shape)
print(kerian_embeddings.shape)
plt.figure(figsize=(16, 8))
plt.imshow(X)

In [ ]:
unknown_embeddings.shape,  embs.shape

In [ ]:
np.cov(X)/

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from exif_tagger.database import FaceDatabase,UNKNOWN_PERSON,Face,Person,Picture
import PIL.Image
import io


imgs = db.getImagesForPerson(pierre.id)
widget_imgs = []
for img in imgs:
    bin = io.BytesIO()
    img.save(bin, format='JPEG')
    widget_imgs.append(widgets.Image(value=bin.getvalue()))



widgets.GridBox(
    children = widget_imgs,
    layout=widgets.Layout(
        grid_template_columns='256px ' * 5,
    )
)

In [ ]:
imgs = db.getImagesForPerson(UNKNOWN_PERSON.id)
imgs

In [ ]:
t = PIL.Image.fromarray(np.array(imgs[0]).astype(np.uint8).transpose(1,2,0))
t

In [ ]:


model = PCA(n_components=2)
model = model.fit(X)

plt.scatter(*model.transform(unknown_embeddings).T, color="b")
plt.scatter(*model.transform(pierre_embeddings_test).T, color="r")
plt.scatter(*model.transform(pierre_embeddings_train).T, color="g")
plt.scatter(*model.transform(people_embeddings).T, color="orange")